# YouTube API: Which videos in a playlist are most popular? 

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np
import os
from googleapiclient.discovery import build

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

#### Import API key from environment

In [5]:
api_key = os.environ.get("YOUTUBE_KEY")

#### Youtube service

In [6]:
youtube = build("youtube", "v3", developerKey=api_key)

#### Define the playlist. In this case we're looking at LAPD's [critical incidents](https://www.youtube.com/playlist?list=PLW5iqZEagvjMvmXRnBaYqozLYwmzUO2B9) releases

In [7]:
playlist_id = "PLW5iqZEagvjMvmXRnBaYqozLYwmzUO2B9"

In [8]:
videos = []

nextPageToken = None

while True:
    pl_request = youtube.playlistItems().list(
        part="contentDetails",
        playlistId=playlist_id,
        maxResults=50,
        pageToken=nextPageToken,
    )

    pl_response = pl_request.execute()

    vid_ids = []
    for item in pl_response["items"]:
        vid_ids.append(item["contentDetails"]["videoId"])

    vid_request = youtube.videos().list(part="statistics", id=",".join(vid_ids))

    vid_response = vid_request.execute()

    for item in vid_response["items"]:
        vid_views = item["statistics"]["viewCount"]
        likes = item["statistics"]["likeCount"]
        vid_id = item["id"]
        yt_link = f"https://youtu.be/{vid_id}"

        videos.append({"views": int(vid_views), "likes": int(likes), "url": yt_link})

    nextPageToken = pl_response.get("nextPageToken")

    if not nextPageToken:
        break

In [9]:
df = pd.DataFrame(videos).sort_values("views", ascending=False)

In [10]:
df.head(20)

,views,likes,url
39,841046,4028,https://youtu.be/bjcdanUhmSY
126,327466,836,https://youtu.be/2yVVn48mYf4
155,297051,450,https://youtu.be/BrBoIWNnbhU
150,214702,1474,https://youtu.be/-Ny7dBpqdOc
139,176559,1616,https://youtu.be/vGU93fhMnfU
114,143541,1501,https://youtu.be/PtSSNn_0GCU
97,136975,1123,https://youtu.be/1hcH7tqKrjY
77,126625,1781,https://youtu.be/2r2UQu6EIt4
154,114417,1108,https://youtu.be/uXPrOZjLfEk
160,106900,1094,https://youtu.be/U8Xpyz1FbQg
